<a href="https://colab.research.google.com/github/surya81/age_gender_detection/blob/main/age_gender_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
def getFile_from_drive( file_id, name ):
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile(name)
getFile_from_drive('1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86',
'fairface-img-margin025-trainval.zip')
getFile_from_drive('1k5vvyREmHDW5TSM9QgB04Bvc8C8_7dl-',
'fairface-label-train.csv')
getFile_from_drive('1_rtz1M1zhvS0d5vVoXUamnohB6cJ02iJ',
'fairface-label-val.csv')
!unzip -qq fairface-img-margin025-trainval.zip

In [2]:
import torch 
import numpy as np ,cv2,pandas as pd,glob,time 
import matplotlib.pyplot as plt 
%matplotlib inline 
import torch.nn as nn 
from torch import optim 
from torch.utils.data import Dataset,DataLoader 
import torch.nn.functional as f 
from torchvision import transforms,models,datasets
device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [3]:
trn_df = pd.read_csv('fairface-label-train.csv')
val_df = pd.read_csv('fairface-label-val.csv')
trn_df.head()

,file,age,gender,race,service_test
0,train/1.jpg,59,Male,East Asian,True
1,train/2.jpg,39,Female,Indian,False
2,train/3.jpg,11,Female,Black,False
3,train/4.jpg,26,Female,Indian,True
4,train/5.jpg,26,Female,Indian,True


In [4]:
from torch.utils.data import Dataset, DataLoader
import cv2
Image_size = 224 
class GenderAgeclassifier(Dataset):
  def __init__(self,df,tfms = None):
    self.df = df 
    self.normalize = transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std = [0.229,0.224,0.225]
    )
  def __len__(self): return len(self.df)
  def __getitem__(self,ix):
    f = self.df.iloc[ix].squeeze()
    file = f.file 
    gen = f.gender =='female'
    age = f.age 
    im = cv2.imread(file)
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    return im,age, gen

  def preprocess_image(self,im):
    im = cv2.resize(im,(Image_size,Image_size))
    im = torch.tensor(im).permute(2,0,1)
    im = self.normalize(im/255)
    return im[None]

  def collate_fn(self,batch):
    ims,ages,genders = [],[],[]
    for im,age,gender in batch:
      im = self.preprocess_image(im)
      ims.append(im)
      ages.append(float(int(age)/80))
      genders.append(float(gender))
    ages,genders = [torch.tensor(x).to(device).float() for x in [ages,genders]]
    ims = torch.cat(ims).to(device)
    return ims,ages,genders


In [5]:
trn = GenderAgeclassifier(trn_df)
val = GenderAgeclassifier(val_df)

In [6]:
trainloader = DataLoader(trn,batch_size = 32,shuffle = True,drop_last =True,collate_fn = trn.collate_fn)
testloader = DataLoader(val,batch_size =32,collate_fn = val.collate_fn)
a,b,c= next(iter(trainloader))
print(a.shape,b.shape,c.shape)

torch.Size([32, 3, 224, 224]) torch.Size([32]) torch.Size([32])


In [7]:
model1 = models.resnet18(pretrained=True).to(device)
from torchsummary import summary
summary(model1, (3,224,224))
model1

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
def get_model():
  model = models.vgg16(pretrained=True)
  for param in model.parameters():
    param.requires_grad = False
  model.avgpool = nn.Sequential(
      nn.Conv2d(512,512,kernel_size= 3),
      nn.MaxPool2d(2),
      nn.ReLU(),
      nn.Flatten()
  )
  class agegenderclassifier(nn.Module):
    def __init__(self):
      super(agegenderclassifier,self).__init__()
      self.intermediate= nn.Sequential(
          nn.Linear(2048,512),
          nn.ReLU(),
          nn.Dropout(0.4),
          nn.Linear(512,128),
          nn.ReLU(),
          nn.Dropout(0.4),
          nn.Linear(128,64),
          nn.ReLU()
      )
      self.age_classifier = nn.Sequential(
          nn.Linear(64,1),
          nn.Sigmoid()
      )
      self.gender_classifier= nn.Sequential(
          nn.Linear(64,1),
          nn.Sigmoid()
      )
    def forward(self,x):
        x = self.intermediate(x)
        age = self.age_classifier(x)
        gender = self.gender_classifier(x)
        return gender , age 
  model.classifier = agegenderclassifier()
  gender_criteriion = nn.BCELoss()
  age__criterion = nn.L1Loss()
  loss_functions = gender_criteriion,age__criterion
  optimizer = torch.optim.Adam(model.parameters(),lr= 1e-3)
  return model.to(device),loss_functions,optimizer


In [9]:
model , criterion , optimizer = get_model()

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [10]:
summary(model,(3,224,224))
model

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
def train_batch(data,model,optimizer,criterion):
  model.train()
  ims,age,gender = data
  optimizer.zero_grad()
  pred_gender,pred_age = model(ims)
  gender_criterion,age_criterion = criterion 
  gender_loss = gender_criterion(pred_gender.squeeze(),gender)
  age_loss = age_criterion(pred_age.squeeze(),age)
  total_loss = gender_loss +age_loss
  total_loss.backward()
  optimizer.step()
  return total_loss

In [12]:
def validate_batch(data,model,criterion):
  model.eval()
  ims, age, gender = data
  with torch.no_grad():
    pred_gender, pred_age = model(ims)
  gender_criterion,age_criterion = criterion
  gender_loss = gender_criterion(pred_gender.squeeze(),gender)
  age_loss = age_criterion(pred_age.squeeze(),age)
  total_loss = gender_loss + age_loss 
  pred_gender = (pred_gender > 0.5).squeeze()
  gender_acc = (pred_gender == gender).float().sum()
  age_mae = torch.abs(age-pred_age).float().sum()
  return total_loss,gender_acc,age_mae


In [13]:
import time 
model , criterion , optimizer = get_model()
val_gender_accuracies = []
val_age_maes = []
train_losses = []
val_losses = []
n_epochs = 5 
best_test_loss = 1000
start = time.time()

for epoch in range(n_epochs):
  epoch_train_loss, epoch_test_loss = 0,0 
  val_age_mae, val_gender_acc,ctr = 0,0,0 
  _n = len(trainloader)
  for ix , data in enumerate(trainloader):
    loss = train_batch(data,model,optimizer,criterion)
    epoch_train_loss += loss.item()

  for ix, data in enumerate(testloader):
    loss,gender_acc,age_mae = validate_batch(data,model,criterion)
    epoch_test_loss += loss.item()
    val_age_mae += age_mae
    val_gender_acc += gender_acc 
    ctr += len(data[0])

  val_age_mae /= ctr
  val_gender_acc /= ctr 
  epoch_train_loss /= len(trainloader)
  epoch_test_loss /= len(testloader)

  elapsed = time.time()-start
  best_test_loss = min(best_test_loss,epoch_test_loss)
  print('{}/{} ({:.2f}s - {:.2f}s remaining)'.format(epoch+1, n_epochs, time.time()-start, (n_epochs-epoch)*(elapsed/(epoch+1))))
  info = f'''Epoch: {epoch+1:03d}\tTrain Loss: {epoch_train_loss:.3f}Test:\{epoch_test_loss:.3f}Best Test Loss: {best_test_loss:.4f}'''
  info += f'\nGender Accuracy:{val_gender_acc*100:.2f}%\tAge MAE: {val_age_mae:.2f}\n'
  print(info)
  val_gender_accuracies.append(val_gender_acc)
  val_age_maes.append(val_age_mae)





/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1/5 (791.34s - 3956.70s remaining)
Epoch: 001	Train Loss: 0.116Test:\0.111Best Test Loss: 0.1113
Gender Accuracy:100.00%	Age MAE: 6.08

2/5 (1585.47s - 3170.94s remaining)
Epoch: 002	Train Loss: 0.098Test:\0.103Best Test Loss: 0.1029
Gender Accuracy:100.00%	Age MAE: 6.64

3/5 (2389.62s - 2389.62s remaining)
Epoch: 003	Train Loss: 0.090Test:\0.099Best Test Loss: 0.0989
Gender Accuracy:100.00%	Age MAE: 6.34

4/5 (3197.54s - 1598.77s remaining)
Epoch: 004	Train Loss: 0.083Test:\0.101Best Test Loss: 0.0989
Gender Accuracy:100.00%	Age MAE: 6.21

5/5 (4012.03s - 802.41s remaining)
Epoch: 005	Train Loss: 0.077Test:\0.100Best Test Loss: 0.0989
Gender Accuracy:100.00%	Age MAE: 5.96



In [ ]:

im = cv2.imread("Image path ")
im = trn.preprocess_image(im).to(device)
gender,age = model(im)
pred_gender = gender.to('cpu').detach().numpy()
pred_age = age.to('cpu').detach().numpy()
im = cv2.imread("image path ")
im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
plt.imshow()
print('predicted gender:',np.where(pred_gender[0][0]<0.5,'Male','Female');'predicted_age', int(pred_age[0][0]*80))
